In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tqdm import tqdm
import matplotlib.pyplot as plt
import pathlib
import os

AUTOTUNE = tf.data.experimental.AUTOTUNE
tf.keras.backend.clear_session()  # For easy reset of notebook state.

tf.__version__
tf.executing_eagerly()


True

In [2]:

ds = tfds.load(name="imagenette", with_info=True, split=["train","validation"])
ds_train=ds[0][0]
ds_test=ds[0][1]
assert isinstance(ds_train, tf.data.Dataset)

def normalize(x):
    y = {'image': tf.image.convert_image_dtype(x['image'], tf.float32), 'label': x['label']}
    y = (tf.image.resize(y['image'], (224,224)), y['label'])
    return y
    
ds_train = ds_train.map(lambda x: normalize(x))
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(100)
ds_train = ds_train.batch(16)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(
    normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(16)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [3]:
class ResNetBlock(tf.keras.layers.Layer):
    def __init__(self, filters=64, stride=1):
        super(ResNetBlock, self).__init__()
        self.stride = stride
        self.conv1 = layers.Conv2D(filters, 3, strides=(self.stride, self.stride), activation='relu', padding='same')
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(filters, 3, padding='same')
        self.bn2 = layers.BatchNormalization()
        if stride == 2:
            self.conv3 = layers.Conv2D(filters,1, strides=(self.stride, self.stride), padding='same' )
            self.bn3 = layers.BatchNormalization()
        self.add1 = layers.Add()
    
    def call(self, inputs, training=False):
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        if self.stride == 2:
            inputs = self.conv3(x)
            inputs = self.bn3(x)
        return tf.nn.relu(self.add1([x, inputs]))
    

class CustomResNetModel(tf.keras.Model):
    def __init__(self):
        super(CustomResNetModel, self).__init__()
        self.conv1 = layers.Conv2D(64, 7, strides=(2, 2), padding='same')
        self.pool1 = layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')
        self.res_block1 = ResNetBlock(64)
        self.res_block2 = ResNetBlock(64)
        self.res_block3 = ResNetBlock(64)
        self.res_block4 = ResNetBlock(128, 2)
        self.res_block5 = ResNetBlock(128)
        self.res_block6 = ResNetBlock(128)
        self.res_block7 = ResNetBlock(128)
        self.res_block8 = ResNetBlock(256, 2)
        self.res_block9 = ResNetBlock(256)
        self.res_block10 = ResNetBlock(256)
        self.res_block11 = ResNetBlock(256)
        self.res_block12 = ResNetBlock(256)
        self.res_block13 = ResNetBlock(256)
        self.res_block14 = ResNetBlock(512, 2)
        self.res_block15 = ResNetBlock(512)
        self.res_block16 = ResNetBlock(512)
        self.pool2 = layers.GlobalAveragePooling2D()
        self.dense1 = layers.Dense(1000, activation='relu')
        self.dense2 = layers.Dense(10, activation='softmax')

    def call(self,inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.res_block1(x)
        x = self.res_block2(x)
        x = self.res_block3(x)
        x = self.res_block4(x)
        x = self.res_block5(x)
        x = self.res_block6(x)
        x = self.res_block7(x)
        x = self.res_block8(x)
        x = self.res_block9(x)
        x = self.res_block10(x)
        x = self.res_block11(x)
        x = self.res_block12(x)
        x = self.res_block13(x)
        x = self.res_block14(x)
        x = self.res_block15(x)
        x = self.res_block16(x)
        x = self.pool2(x)
        x = self.dense1(x)
        
        return self.dense2(x)

In [5]:
model = CustomResNetModel()

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.01),
    metrics=['accuracy'],
)


In [ ]:
model.fit(
    x=ds_train,
    #steps_per_epoch=1,
    epochs=5,
    validation_data=ds_test,
)
model.run_eagerly = False

Epoch 1/5
806/806 [==============================] - 171s 212ms/step - loss: 1.5170 - accuracy: 0.4897 - val_loss: 2.2478 - val_accuracy: 0.3080
Epoch 2/5
806/806 [==============================] - 171s 212ms/step - loss: 1.3659 - accuracy: 0.5491 - val_loss: 2.1777 - val_accuracy: 0.3600
Epoch 3/5
806/806 [==============================] - 171s 213ms/step - loss: 1.2417 - accuracy: 0.5895 - val_loss: 1.9350 - val_accuracy: 0.4280
Epoch 4/5
757/806 [===========================>..] - ETA: 10s - loss: 1.1057 - accuracy: 0.6386

In [ ]:
model.fit(
    x=ds_train,
    #steps_per_epoch=1,
    epochs=5,
    validation_data=ds_test,
)
model.run_eagerly = False

In [ ]:
model.fit(
    x=ds_train,
    #steps_per_epoch=1,
    epochs=5,
    validation_data=ds_test,
)
model.run_eagerly = False

In [ ]:
model.fit(
    x=ds_train,
    #steps_per_epoch=1,
    epochs=5,
    validation_data=ds_test,
)
model.run_eagerly = False

In [ ]:
what